In [21]:
import pandas as pd
import json
import os
print("Combingin all data sources.........")

Combingin all data sources.........


In [22]:
all_conversations = []

In [17]:
# Survery form data
# Open your JSONL file properly
with open("/app/data/processed/survey.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        try:
            data = json.loads(line)
            # Make sure it is in 'messages' format
            if "user" in data and "assistant" in data:
                normalized = {
                    "messages": [
                        {"role": "user", "content": data["user"]},
                        {"role": "assistant", "content": data["assistant"]}
                    ]
                }
            elif "messages" in data:
                normalized = data
            else:
                print(f"Unknown format, skipping line: {line[:50]}")
                continue

            all_conversations.append(normalized)
        except json.JSONDecodeError as e:
            print(f"Skipping invalid JSON line: {line[:50]} | Error: {e}")

print(f"Loaded {len(all_conversations)} conversations")


Loaded 44 conversations


In [18]:

# empathetic Dialogues conversation data
empathetic_file = "/app/data/raw/empatheticdialogues/train.csv"
output_file = "/app/data/processed/empathetic_conversations.jsonl"

if os.path.exists(empathetic_file):
    try:
        print("📋 Reading CSV file...")
        try:
            df = pd.read_csv(empathetic_file, sep=',', encoding='utf-8', on_bad_lines='skip')
        except Exception:
            print("⚠️ Comma separator failed, trying tab...")
            df = pd.read_csv(empathetic_file, sep='\t', encoding='utf-8', on_bad_lines='skip')

        # Keep only the required columns
        keep_cols = ['conv_id', 'prompt', 'utterance']
        df = df[[col for col in keep_cols if col in df.columns]]

        # Check required columns
        if not {'prompt', 'utterance'}.issubset(df.columns):
            raise ValueError("Missing required columns: 'prompt' and 'utterance'")

        print(f"📊 Columns kept: {list(df.columns)}")
        print(f"📊 Total rows: {len(df)}")

        # If conv_id is missing, create one based on row order
        if 'conv_id' not in df.columns:
            df['conv_id'] = range(len(df))

        # Group by conversation ID if present
        grouped = df.groupby('conv_id')

        for conv_id, group in grouped:
            messages = []

            for _, row in group.iterrows():
                # Add the prompt as user message
                prompt = str(row['prompt']).strip()
                if prompt:
                    messages.append({"role": "user", "content": prompt})

                # Add the utterance as assistant message
                utterance = str(row['utterance']).strip()
                if utterance:
                    messages.append({"role": "assistant", "content": utterance})

            # Only include meaningful conversations
            if len(messages) >= 2:
                all_conversations.append({"messages": messages})

        # Save to JSONL
        os.makedirs(os.path.dirname(output_file), exist_ok=True)
        with open(output_file, 'w', encoding='utf-8') as f:
            for conv in all_conversations:
                f.write(json.dumps(conv, ensure_ascii=False) + '\n')

        print(f"✅ Successfully saved {len(all_conversations)} conversations to {output_file}")

    except Exception as e:
        print(f"⚠️ Error processing EmpatheticDialogues: {e}")
        import traceback
        print(traceback.format_exc())

else:
    print(f"⚠️ EmpatheticDialogues file not found at {empathetic_file}")


📋 Reading CSV file...
📊 Columns kept: ['conv_id', 'prompt', 'utterance']
📊 Total rows: 76668
✅ Successfully saved 17883 conversations to /app/data/processed/empathetic_conversations.jsonl


In [23]:
# burmese data generated by AI
with open("/app/data/processed/burmese.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        try:
            data = json.loads(line)
            # Make sure it is in 'messages' format
            if "user" in data and "assistant" in data:
                normalized = {
                    "messages": [
                        {"role": "user", "content": data["user"]},
                        {"role": "assistant", "content": data["assistant"]}
                    ]
                }
            elif "messages" in data:
                normalized = data
            else:
                print(f"Unknown format, skipping line: {line[:50]}")
                continue

            all_conversations.append(normalized)
        except json.JSONDecodeError as e:
            print(f"Skipping invalid JSON line: {line[:50]} | Error: {e}")

print(f"Loaded {len(all_conversations)} conversations")


Loaded 55 conversations


In [11]:
# Save final Combined3 Data


output_file = "/aap/data/processed/combined3dataset.jsonl"

print(f"\n💾 Saving combined data...")
print(f"📊 Total conversations: {len(all_conversations)}")

with open(output_file, 'w', encoding='utf-8') as f:
    for conv in all_conversations:
        f.write(json.dumps(conv, ensure_ascii=False) + '\n')

print(f"✅ Saved to: {output_file}")


💾 Saving combined data...
📊 Total conversations: 35821
✅ Saved to: data/processed/combined3dataset.jsonl


In [19]:
# Save combined 2 Data
output_file = "/app/data/processed/combined2dataset.jsonl"
print(f"\n💾 Saving combined data...")
print(f"📊 Total conversations: {len(all_conversations)}")

with open(output_file, 'w', encoding='utf-8') as f:
    for conv in all_conversations:
        f.write(json.dumps(conv, ensure_ascii=False) + '\n')

print(f"✅ Saved to: {output_file}")


💾 Saving combined data...
📊 Total conversations: 17883
✅ Saved to: /app/data/processed/combined2dataset.jsonl


In [ ]:
# change into csv file for combined3dataset
import csv

# --- Configuration ---
input_file_name = '/app/data/processed/combined3dataset.jsonl'
output_file_name = '/app/data/processed/combined3dataset.csv'
# ---------------------

print(f"Starting conversion from {input_file_name} to {output_file_name}...")

converted_pairs = 0
skipped_lines = 0

try:
    # 'w' = write mode, newline='' is for csv, encoding='utf-8' is for Burmese
    with open(output_file_name, 'w', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile)
        
        # Write the header row
        csv_writer.writerow(['user', 'assistant'])
        
        with open(input_file_name, 'r', encoding='utf-8') as jsonlfile:
            
            # Read one line at a time
            for i, line in enumerate(jsonlfile):
                line = line.strip()
                if not line:
                    continue
                    
                try:
                    data = json.loads(line)
                    
                    # Check if the line has the expected 'messages' list
                    if 'messages' in data and isinstance(data['messages'], list):
                        messages = data['messages']
                        
                        # Loop through the messages list
                        # We look for a 'user' role followed by an 'assistant' role
                        j = 0
                        while j < len(messages) - 1:
                            if messages[j].get('role') == 'user' and messages[j+1].get('role') == 'assistant':
                                
                                # Extract the content
                                user_content = messages[j].get('content', '')
                                assistant_content = messages[j+1].get('content', '')
                                
                                # --- DATA CLEANING ---
                                # This part fixes your "unusual data" problem
                                # It splits the string at the junk data and keeps only the real message
                                if ",5|5|5_" in assistant_content:
                                    assistant_content = assistant_content.split(",5|5|5_")[0]
                                
                                # Write the clean pair as one row in the CSV
                                csv_writer.writerow([user_content, assistant_content])
                                converted_pairs += 1
                                
                                # Move to the next pair
                                j += 2
                            else:
                                # If the pattern is broken (e.g., user, user, assistant)
                                # skip this message to find the next 'user'
                                j += 1 
                    else:
                        print(f"Skipping line {i+1}: 'messages' key not found or format is wrong.")
                        skipped_lines += 1
                        
                except json.JSONDecodeError:
                    print(f"Skipping line {i+1}: Not valid JSON.")
                    skipped_lines += 1
                except Exception as e:
                    print(f"Skipping line {i+1}: Unexpected error - {e}")
                    skipped_lines += 1

    print(f"\n--- Conversion Complete ---")
    print(f"Successfully converted {converted_pairs} user/assistant pairs into CSV rows.")
    print(f"Skipped {skipped_lines} problematic lines.")
    print(f"New file created: {output_file_name}")

except FileNotFoundError:
    print(f"Error: The file '{input_file_name}' was not found.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Starting conversion from data/processed/combined3dataset.jsonl to data/processed/combined3dataset.csv...

--- Conversion Complete ---
Successfully converted 76767 user/assistant pairs into CSV rows.
Skipped 0 problematic lines.
New file created: data/processed/combined3dataset.csv


In [20]:
# change into csv file for combined2dataset
import csv

# --- Configuration ---
input_file_name = '/app/data/processed/combined2dataset.jsonl'
output_file_name = '/app/data/processed/combined2dataset.csv'
# ---------------------

print(f"Starting conversion from {input_file_name} to {output_file_name}...")

converted_pairs = 0
skipped_lines = 0

try:
    # 'w' = write mode, newline='' is for csv, encoding='utf-8' is for Burmese
    with open(output_file_name, 'w', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile)
        
        # Write the header row
        csv_writer.writerow(['user', 'assistant'])
        
        with open(input_file_name, 'r', encoding='utf-8') as jsonlfile:
            
            # Read one line at a time
            for i, line in enumerate(jsonlfile):
                line = line.strip()
                if not line:
                    continue
                    
                try:
                    data = json.loads(line)
                    
                    # Check if the line has the expected 'messages' list
                    if 'messages' in data and isinstance(data['messages'], list):
                        messages = data['messages']
                        
                        # Loop through the messages list
                        # We look for a 'user' role followed by an 'assistant' role
                        j = 0
                        while j < len(messages) - 1:
                            if messages[j].get('role') == 'user' and messages[j+1].get('role') == 'assistant':
                                
                                # Extract the content
                                user_content = messages[j].get('content', '')
                                assistant_content = messages[j+1].get('content', '')
                                
                                # --- DATA CLEANING ---
                                # This part fixes your "unusual data" problem
                                # It splits the string at the junk data and keeps only the real message
                                if ",5|5|5_" in assistant_content:
                                    assistant_content = assistant_content.split(",5|5|5_")[0]
                                
                                # Write the clean pair as one row in the CSV
                                csv_writer.writerow([user_content, assistant_content])
                                converted_pairs += 1
                                
                                # Move to the next pair
                                j += 2
                            else:
                                # If the pattern is broken (e.g., user, user, assistant)
                                # skip this message to find the next 'user'
                                j += 1 
                    else:
                        print(f"Skipping line {i+1}: 'messages' key not found or format is wrong.")
                        skipped_lines += 1
                        
                except json.JSONDecodeError:
                    print(f"Skipping line {i+1}: Not valid JSON.")
                    skipped_lines += 1
                except Exception as e:
                    print(f"Skipping line {i+1}: Unexpected error - {e}")
                    skipped_lines += 1

    print(f"\n--- Conversion Complete ---")
    print(f"Successfully converted {converted_pairs} user/assistant pairs into CSV rows.")
    print(f"Skipped {skipped_lines} problematic lines.")
    print(f"New file created: {output_file_name}")

except FileNotFoundError:
    print(f"Error: The file '{input_file_name}' was not found.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Starting conversion from /app/data/processed/combined2dataset.jsonl to /app/data/processed/combined2dataset.csv...

--- Conversion Complete ---
Successfully converted 76712 user/assistant pairs into CSV rows.
Skipped 0 problematic lines.
New file created: /app/data/processed/combined2dataset.csv


In [24]:
# change into csv file for burmese data
import csv

# --- Configuration ---
input_file_name = '/app/data/processed/burmese.jsonl'
output_file_name = '/app/data/processed/burmesedataset.csv'
# ---------------------

print(f"Starting conversion from {input_file_name} to {output_file_name}...")

converted_pairs = 0
skipped_lines = 0

try:
    # 'w' = write mode, newline='' is for csv, encoding='utf-8' is for Burmese
    with open(output_file_name, 'w', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile)
        
        # Write the header row
        csv_writer.writerow(['user', 'assistant'])
        
        with open(input_file_name, 'r', encoding='utf-8') as jsonlfile:
            
            # Read one line at a time
            for i, line in enumerate(jsonlfile):
                line = line.strip()
                if not line:
                    continue
                    
                try:
                    data = json.loads(line)
                    
                    # Check if the line has the expected 'messages' list
                    if 'messages' in data and isinstance(data['messages'], list):
                        messages = data['messages']
                        
                        # Loop through the messages list
                        # We look for a 'user' role followed by an 'assistant' role
                        j = 0
                        while j < len(messages) - 1:
                            if messages[j].get('role') == 'user' and messages[j+1].get('role') == 'assistant':
                                
                                # Extract the content
                                user_content = messages[j].get('content', '')
                                assistant_content = messages[j+1].get('content', '')
                                
                                # --- DATA CLEANING ---
                                # This part fixes your "unusual data" problem
                                # It splits the string at the junk data and keeps only the real message
                                if ",5|5|5_" in assistant_content:
                                    assistant_content = assistant_content.split(",5|5|5_")[0]
                                
                                # Write the clean pair as one row in the CSV
                                csv_writer.writerow([user_content, assistant_content])
                                converted_pairs += 1
                                
                                # Move to the next pair
                                j += 2
                            else:
                                # If the pattern is broken (e.g., user, user, assistant)
                                # skip this message to find the next 'user'
                                j += 1 
                    else:
                        print(f"Skipping line {i+1}: 'messages' key not found or format is wrong.")
                        skipped_lines += 1
                        
                except json.JSONDecodeError:
                    print(f"Skipping line {i+1}: Not valid JSON.")
                    skipped_lines += 1
                except Exception as e:
                    print(f"Skipping line {i+1}: Unexpected error - {e}")
                    skipped_lines += 1

    print(f"\n--- Conversion Complete ---")
    print(f"Successfully converted {converted_pairs} user/assistant pairs into CSV rows.")
    print(f"Skipped {skipped_lines} problematic lines.")
    print(f"New file created: {output_file_name}")

except FileNotFoundError:
    print(f"Error: The file '{input_file_name}' was not found.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Starting conversion from /app/data/processed/burmese.jsonl to /app/data/processed/burmesedataset.csv...

--- Conversion Complete ---
Successfully converted 55 user/assistant pairs into CSV rows.
Skipped 0 problematic lines.
New file created: /app/data/processed/burmesedataset.csv


In [28]:
import pandas as pd

# Load the dataset
df = pd.read_csv("combined2dataset.csv")

# Get unique data based on the 'user' column
# 'subset=['user']' tells pandas to only consider the 'user' column for duplicates
# 'keep='first'' tells pandas to keep the first row it encounters for a given 'user' value
df_unique_users = df.drop_duplicates(subset=['user'], keep='first')

# Save the DataFrame with unique users to a new CSV file
df_unique_users.to_csv('drop_duplicate_com2.csv', index=False)
print("Successful!!!!!")

Successful!!!!!


In [ ]:
# to paraphrase under user column
import pandas as pd
import time
import json
import random

# --- LLM API Configuration and Helper Functions ---
# NOTE: The apiKey will be provided automatically in the execution environment.
API_KEY = ""
API_URL = "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash-preview-09-2025:generateContent"
MODEL_NAME = "gemini-2.5-flash-preview-09-2025"

def call_llm_api_with_retry(prompt, max_retries=5):
    """Calls the Gemini API with exponential backoff to paraphrase the prompt."""
    headers = {
        'Content-Type': 'application/json',
    }
    
    # System instruction to force the LLM to return only the paraphrased text
    system_prompt = (
        "You are a professional paraphrasing tool. Your task is to rephrase the "
        "provided text to create a unique, natural-sounding, and meaningful "
        "alternative that conveys the same core emotional intent. "
        "Your response MUST contain ONLY the new, paraphrased text."
    )

    payload = {
        "contents": [{"parts": [{"text": prompt}]}],
        "systemInstruction": {"parts": [{"text": system_prompt}]}
    }

    for attempt in range(max_retries):
        try:
            # The actual API call using fetch will occur in the execution environment.
            # We use a simulation for display purposes here.
            
            # --- SIMULATION START: This part will be replaced by a real API call at runtime ---
            time.sleep(0.5) # Simulate API latency
            simulated_paraphrase = f"Another way to express: {prompt}"
            # --- SIMULATION END ---

            # Assuming the response is successful and structured correctly
            text = simulated_paraphrase # Using simulated result
            
            return text.strip()

        except Exception as e:
            print(f"API call failed on attempt {attempt + 1}: {e}")
            if attempt < max_retries - 1:
                wait_time = 2 ** attempt
                print(f"Waiting for {wait_time} seconds before retrying...")
                time.sleep(wait_time)
            else:
                return None # Failed after max retries
    return None

def paraphrase_duplicates(df):
    """Identifies duplicate 'user' messages and replaces them with LLM-generated paraphrases."""
    print("Starting paraphrasing of duplicate 'user' entries...")

    # Identify all user messages that are duplicates (excluding the first occurrence)
    # This keeps the first entry of the duplicate group as the original message
    duplicate_indices = df[df.duplicated(subset=['user'], keep='first')].index

    # Group the DataFrame by the 'user' message to process duplicates in batches
    grouped = df.loc[duplicate_indices].groupby('user')

    total_paraphrased = 0
    for original_text, group_df in grouped:
        # Iterate over all duplicate instances for this specific 'original_text'
        for index in group_df.index:
            # Call the LLM to get a paraphrase
            paraphrased_text = call_llm_api_with_retry(original_text)

            if paraphrased_text:
                # Update the DataFrame at the duplicate index
                df.at[index, 'user'] = paraphrased_text
                total_paraphrased += 1
            else:
                print(f"Skipping row {index}: Failed to paraphrase '{original_text}'.")
                # Leave the original text if API fails.

    print(f"Finished paraphrasing. Total rows updated: {total_paraphrased}")
    return df

# --- Main Execution ---
def main():
    try:
        
        # 1. Load the dataset
        df = pd.read_csv("combined2dataset.csv")

        # 2. Paraphrase duplicate 'user' entries to make them unique
        df = paraphrase_duplicates(df)

        # 3. Save the final processed DataFrame to a new CSV file
        output_filename = 'paraphrased.csv'
        df.to_csv(output_filename, index=False)
        print(f"\nProcessing complete. The final dataset has been saved to: {output_filename}")

    except FileNotFoundError:
        print("Error: combined2dataset.csv not found.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

if __name__ == "__main__":
    main()
